In [5]:
import os
from dotenv import load_dotenv, find_dotenv
from langchain_core.messages import SystemMessage, HumanMessage
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_openai import ChatOpenAI
from IPython.display import display, HTML
from langchain.chains import LLMChain, SequentialChain

load_dotenv(find_dotenv(), override=True)

True

In [6]:
# CHAIN 1:
# takes info about the student and generates a timetable in plaintext

system_message = '''
You are an assistant that creates personalized study plans for students. You always have to take into account data \
specific to a student, and ensure that the timetable you make caters to the student's needs and requirements. 
'''

num_weeks = 2

student_data = {
    "name": "Jason Statham",
    "field_of_study": "Computer Science",
    "year_of_study": "2nd Year",
    "subjects": "Data Structures, Algorithms, Computer Architecture, Discrete Mathematics",
    "learning_styles": "Visual, Kinesthetic",
    "personal_objectives": "Prepare for upcoming internship interview, improve coding skills, and practice guitar",
    "challenges": "Some difficulty in understanding theory",
    "extracurricular_activities": "Football player, guitarist"
}

sample_timetable = '''
### Week 1

#### Monday
- **9:00 AM - 10:30 AM**: Data Structures (Visual aids, flowcharts)
- **10:30 AM - 11:00 AM**: Break
- **11:00 AM - 12:30 PM**: Algorithms (Hands-on coding practice)
- **12:30 PM - 1:30 PM**: Lunch
- **1:30 PM - 3:00 PM**: Discrete Mathematics (Interactive activities)
- **3:00 PM - 3:30 PM**: Break
- **3:30 PM - 5:00 PM**: Computer Architecture (Visual diagrams, models)
- **5:00 PM - 6:00 PM**: Review and Summary (Flashcards, visual summaries)
- **6:00 PM onwards**: Soccer practice

#### Tuesday
- **9:00 AM - 10:30 AM**: Algorithms (Problem-solving sessions)
- **10:30 AM - 11:00 AM**: Break
- **11:00 AM - 12:30 PM**: Data Structures (Group study, discussions)
- **12:30 PM - 1:30 PM**: Lunch
- **1:30 PM - 3:00 PM**: Discrete Mathematics (Kinesthetic learning, puzzles)
- **3:00 PM - 3:30 PM**: Break
- **3:30 PM - 5:00 PM**: Computer Architecture (Interactive simulations)
- **5:00 PM - 6:00 PM**: Review and Summary (Mind maps)
- **6:00 PM onwards**: Volunteering at local NGO
'''

user_template = '''
You are required to make a timetable for a student, based on their needs and preferences. 
You are being provided with information about a student, and the number of weeks for which the timetable must be generated. 
You are also being provided the format you should follow from a sample timetable of 2 days. 

Number of weeks: {num_weeks}
Student information (delimited by triple back ticks): ```{student_data}```
Sample timetable (delimited by triple back ticks): ```{sample_timetable}```

Please generate the timetable in plaintext format, following the style of the sample timetable.
'''

# creating the prompt for the first chain
user_template_prompt = HumanMessagePromptTemplate.from_template(user_template)

first_chat_template = ChatPromptTemplate.from_messages(
    [
        system_message, 
        user_template_prompt
    ]
)

# creating the first chain
llm1 = ChatOpenAI(model='gpt-4o', temperature=0.5)
chain1 = LLMChain(llm=llm1, prompt=first_chat_template, output_key='plaintext_timetable')



In [7]:
# CHAIN 2:
# takes the plaintext timetable and converts it to an HTML table (for easier viewing)

html_sample = '''
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Timetable</title>
</head>
<body>
    <h2>Week 1</h2>

    <h3>Monday</h3>
    <ul>
        <li><strong>9:00 AM - 10:30 AM</strong>: Data Structures (Visual aids, flowcharts)</li>
        <li><strong>10:30 AM - 11:00 AM</strong>: Break</li>
        <li><strong>11:00 AM - 12:30 PM</strong>: Algorithms (Hands-on coding practice)</li>
        <li><strong>12:30 PM - 1:30 PM</strong>: Lunch</li>
        <li><strong>1:30 PM - 3:00 PM</strong>: Discrete Mathematics (Interactive activities)</li>
        <li><strong>3:00 PM - 3:30 PM</strong>: Break</li>
        <li><strong>3:30 PM - 5:00 PM</strong>: Computer Architecture (Visual diagrams, models)</li>
        <li><strong>5:00 PM - 6:00 PM</strong>: Review and Summary (Flashcards, visual summaries)</li>
        <li><strong>6:00 PM onwards</strong>: Football practice</li>
    </ul>

    <h3>Tuesday</h3>
    <ul>
        <li><strong>9:00 AM - 10:30 AM</strong>: Algorithms (Problem-solving sessions)</li>
        <li><strong>10:30 AM - 11:00 AM</strong>: Break</li>
        <li><strong>11:00 AM - 12:30 PM</strong>: Data Structures (Group study, discussions)</li>
        <li><strong>12:30 PM - 1:30 PM</strong>: Lunch</li>
        <li><strong>1:30 PM - 3:00 PM</strong>: Discrete Mathematics (Kinesthetic learning, puzzles)</li>
        <li><strong>3:00 PM - 3:30 PM</strong>: Break</li>
        <li><strong>3:30 PM - 5:00 PM</strong>: Computer Architecture (Interactive simulations)</li>
        <li><strong>5:00 PM - 6:00 PM</strong>: Review and Summary (Mind maps)</li>
        <li><strong>6:00 PM onwards</strong>: Guitar practice</li>
    </ul>
'''

user_template2 = '''
You will be provided with a timetable for a student in plaintext. You are required to convert this timetable to an HTML format. \
Do not change the contents of the timetable. 
You are also being provided the HTML table format you should follow from a sample timetable of 2 days. 

Plaintext timetable (delimited by triple backticks): ```{plaintext_timetable}```
Sample HTML format (delimited by triple backticks): ```{html_sample}```

Please output ONLY the HTML code for the table, and nothing else. 
'''

user_template_prompt2 = HumanMessagePromptTemplate.from_template(user_template2)

second_chat_template = ChatPromptTemplate.from_messages([user_template_prompt2])

llm2 = ChatOpenAI(model='gpt-4o', temperature=0.3)
# llm2 = ChatOpenAI(model='gpt-3.5-turbo', temperature=0.3)
chain2 = LLMChain(llm=llm2, prompt=second_chat_template, output_key='html_timetable')

In [8]:
# making and running the overall chain

overall_chain = SequentialChain(
    chains=[chain1, chain2],
    input_variables=['num_weeks', 'student_data', 'sample_timetable', 'html_sample'],
    output_variables=['plaintext_timetable', 'html_timetable']
)

chain_outputs = overall_chain.invoke({'num_weeks': num_weeks, 'student_data': student_data, 'sample_timetable': sample_timetable, 'html_sample': html_sample})

# getting the html output for displaying it
html_output = chain_outputs['html_timetable']

# saving the html data to view the timetable in a web browser (or to analyze the raw html if needed)
with open('timetable.html', 'w', encoding='utf-8') as f: 
    f.write(html_output)

display(HTML(html_output))